In [5]:
import os
import sys
from datetime import datetime, timezone, timedelta
from urllib.request import urlopen
from urllib.error import HTTPError
import logging
from bs4 import BeautifulSoup
from sqlalchemy import exc, create_engine
import pymysql
import numpy as np
import pandas as pd
import boto3
from botocore.exceptions import ClientError
# import pyarrow
import awswrangler as wr
# from urllib.error import URLError, HTTPError
from utils import *
# from praw.models import MoreComments
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import sentry_sdk

logging.basicConfig(filename='example.log', level=logging.DEBUG, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.info('Starting Logging Function')

today = datetime.now().date()
yesterday = today - timedelta(1)
day = (datetime.now() - timedelta(1)).day
month = (datetime.now() - timedelta(1)).month
year = (datetime.now() - timedelta(1)).year
season_type = 'Regular Season'

sentry_sdk.init(
    "https://2086ed87baf1469e8b4b4f889dfbd088@o1007718.ingest.sentry.io/6259006",

    # Set traces_sample_rate to 1.0 to capture 100%
    # of transactions for performance monitoring.
    # We recommend adjusting this value in production.
    traces_sample_rate=1.0
)

ModuleNotFoundError: No module named 'praw'

In [4]:
data1 = df[0].copy()
data1.columns.values[0] = "Tomorrow"
date_try = str(year) + " " + data1.columns[0]
data1["date"] = np.where(
    date_try == "2022 Tomorrow",
    datetime.now().date(),  # if the above is true, then return this
    str(year) + " " + data1.columns[0],  # if false then return this
)
# )
date_try = data1["date"].iloc[0]
data1.reset_index(drop=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace(
    "LA Clippers", "LAC Clippers", regex=True
)

data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
data1["Time"] = data1["Tomorrow"].str.split().str[0]
data1["datetime1"] = (
    pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
    - timedelta(hours=6)
    + timedelta(days=1)
)
if len(df) > 1:  # if more than 1 day's data appears then do this
    data2 = df[1].copy()
    data2.columns.values[0] = "Tomorrow"
    data2.reset_index(drop=True)
    data2["Tomorrow"] = data2["Tomorrow"].str.replace(
        "LA Clippers", "LAC Clippers", regex=True
    )
    data2["Tomorrow"] = data2["Tomorrow"].str.replace("AM", "AM ", regex=True)
    data2["Tomorrow"] = data2["Tomorrow"].str.replace("PM", "PM ", regex=True)
    data2["Time"] = data2["Tomorrow"].str.split().str[0]
    data2["datetime1"] = (
        pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data2["Time"])
        - timedelta(hours=6)
        + timedelta(days=1)
    )
    data2["date"] = data2["datetime1"].dt.date
schedule_df.to_sql(con = conn, name = "aws_schedule_source", if_exists = 'replace', index = False)
            for line in data["Tomorrow"]
        ]
    )
    data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
    data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
    data.columns = data.columns.str.lower()
    data = data[
        ["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]
    ]
    data = data.rename(columns={data.columns[0]: "team"})
    data = data.query("date == date.min()")  # only grab games from upcoming day

In [2]:
reddit = praw.Reddit(
    client_id=os.environ.get("reddit_accesskey"),
    client_secret=os.environ.get("reddit_secretkey"),
    user_agent="praw-app",
    username=os.environ.get("reddit_user"),
    password=os.environ.get("reddit_pw"),
)

In [3]:
def get_reddit_data(sub):
    """
    Web Scrape function w/ PRAW that grabs top ~27 top posts from a given subreddit.
    Left sub as an argument in case I want to scrape multi subreddits in the future (r/nba, r/nbadiscussion, r/sportsbook etc)
    Args:
        sub (string) - subreddit to query
    Returns:
        Pandas DataFrame of all current top posts on r/nba
    """
    reddit = praw.Reddit(
        client_id=os.environ.get("reddit_accesskey"),
        client_secret=os.environ.get("reddit_secretkey"),
        user_agent="praw-app",
        username=os.environ.get("reddit_user"),
        password=os.environ.get("reddit_pw"),
    )
    try:
        subreddit = reddit.subreddit(sub)
        posts = []
        for post in subreddit.hot(limit=27):
            posts.append(
                [
                    post.title,
                    post.score,
                    post.id,
                    post.url,
                    str(f'https://www.reddit.com{post.permalink}'),
                    post.num_comments,
                    post.selftext,
                    today,
                    todaytime,
                ]
            )
        posts = pd.DataFrame(
            posts,
            columns=[
                "title",
                "score",
                "id",
                "url",
                "reddit_url",
                "num_comments",
                "body",
                "scrape_date",
                "scrape_time",
            ],
        )
        posts.columns = posts.columns.str.lower()

        print(
            f"Reddit Scrape Successful, grabbing 27 Recent popular posts from r/{sub} subreddit"
        )
        logging.info(
            f"Reddit Scrape Successful, grabbing 27 Recent popular posts from r/{sub} subreddit"
        )
        return posts
    except BaseException as error:
        logging.info(f"Reddit Scrape Function Failed, {error}")
        print(f"Reddit Scrape Function Failed, {error}")
        data = []
        return data

In [4]:
reddit_data = get_reddit_data("nba")

Reddit Scrape Successful, grabbing 27 Recent popular posts from r/nba subreddit


In [6]:
subreddit = reddit.subreddit('nba')
for post in subreddit.hot(limit=1):
    print(vars(post))

{'comment_limit': 2048, 'comment_sort': 'confidence', '_reddit': <praw.reddit.Reddit object at 0x7fa77b9e9040>, 'approved_at_utc': None, 'subreddit': Subreddit(display_name='nba'), 'selftext': '# Game Threads Index (January 03, 2022):\n\n|Tip-off|GDT|Away|Score|Home|PGT|\n|:--|:--:|:--|:-:|:--|:--:|\n|7:00 PM ET|[Link](https://www.reddit.com/r/nba/comments/rvemxr/game_thread_houston_rockets_1027_philadelphia/)|[Houston Rockets](/r/rockets)| 1Q >!0 to 0!<|[Philadelphia 76ers](/r/sixers)||\n|7:00 PM ET|[Link](https://www.reddit.com/r/nba/comments/rvemxh/game_thread_charlotte_hornets_1918_washington/)|[Charlotte Hornets](/r/charlottehornets)| 0Q >! to !<|[Washington Wizards](/r/washingtonwizards)||\n|7:30 PM ET|[Link](https://www.reddit.com/r/nba/comments/rvfay1/game_thread_memphis_grizzlies_2314_brooklyn_nets/)|[Memphis Grizzlies](/r/memphisgrizzlies)|PRE-GAME|[Brooklyn Nets](/r/gonets)||\n|8:00 PM ET|[Link](https://www.reddit.com/r/nba/comments/rvfy1f/game_thread_orlando_magic_730_chica

In [23]:
urls = reddit_data['reddit_url']
urls2 = urls[urls.str.contains('reddit')]

In [30]:
def get_reddit_comments(url):
    author_list = []
    flair_list1 = []
    flair_list2 = []
    comment_list = []
    score_list = []
    url_list = []
    edited_list = []
    try:
        for i in url:
            submission = reddit.submission(url = i)
            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                author_list.append(comment.author)
                flair_list1.append(comment.author_flair_css_class)
                flair_list2.append(comment.author_flair_text)
                comment_list.append(comment.body)
                score_list.append(comment.score)
                edited_list.append(comment.edited)
                url_list.append(i)
        df = pd.DataFrame({'comment':comment_list, 'score':score_list, 'url':url_list,
             'author': author_list, 'flair1': flair_list1, 'flair2': flair_list2, 'edited': edited_list,
              'scrape_date': datetime.now().date(), 'scrape_ts': datetime.now()})
        df = df.astype({"author": str})
        # adding sentiment analysis columns
        analyzer = SentimentIntensityAnalyzer()
        df["compound"] = [
            analyzer.polarity_scores(x)["compound"] for x in df["comment"]
        ]
        df["neg"] = [analyzer.polarity_scores(x)["neg"] for x in df["comment"]]
        df["neu"] = [analyzer.polarity_scores(x)["neu"] for x in df["comment"]]
        df["pos"] = [analyzer.polarity_scores(x)["pos"] for x in df["comment"]]
        df["sentiment"] = np.where(df["compound"] > 0, 1, 0)
        
        print(f'Reddit Comment Extraction Success, retrieving {len(df)} total comments from {len(url)} total urls')
        logging.info(f'Reddit Comment Extraction Success, retrieving {len(df)} total comments from {len(url)} total urls')
        return df
    except BaseException as e:
        print(f'Reddit Comment Extraction Failed for url {i}, {e}')
        logging.info(f'Reddit Comment Extraction Failed for url {i}, {e}')
        df = []
        return df

In [31]:
df = get_reddit_comments(urls2)

Reddit Comment Extraction Success, retrieving 4778 total comments from 27 total urls


In [34]:
df = df.astype({"author": str})

In [35]:
df.to_sql(con = conn, name = 'aws_reddit_comment_data_source', if_exists = 'replace', index = False)

In [32]:
df

,comment,score,url,author,flair1,flair2,edited,scrape_date,scrape_ts,compound,neg,neu,pos,sentiment
0,[Active COVID-19 cases in the NBA](https://old...,1,https://www.reddit.com/r/nba/comments/rv16lf/d...,howser343,Bulls1,[CHI] Patrick Williams,False,2022-01-03,2022-01-03 18:47:46.692418,0.0000,0.000,1.000,0.000,0
1,I was reading Anthony Bennett's thread (waived...,13,https://www.reddit.com/r/nba/comments/rv16lf/d...,Rahnamatta,Heat3,Heat,False,2022-01-03,2022-01-03 18:47:46.692418,0.3818,0.089,0.799,0.111,1
2,Where does Kevin Love rank in the MIP race,4,https://www.reddit.com/r/nba/comments/rv16lf/d...,macknuggets,Nuggets5,:den-5: Nuggets,False,2022-01-03,2022-01-03 18:47:46.692418,0.6369,0.000,0.656,0.344,1
3,"Any updates on Ben Simmons status, beyond trad...",3,https://www.reddit.com/r/nba/comments/rv16lf/d...,-50k-,None,None,False,2022-01-03,2022-01-03 18:47:46.692418,0.0000,0.000,1.000,0.000,0
4,Hello,2,https://www.reddit.com/r/nba/comments/rv16lf/d...,indian_hannibal,Lakers1,[LAL] Magic Johnson,False,2022-01-03,2022-01-03 18:47:46.692418,0.0000,0.000,1.000,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,Fuck Woj,2,https://www.reddit.com/r/nba/comments/rv7yrd/w...,Salsa_Z5,Bucks2,Bucks,False,2022-01-03,2022-01-03 18:47:46.692418,-0.5423,0.778,0.222,0.000,0
4774,that man was hitting strip clubs after playoff...,2,https://www.reddit.com/r/nba/comments/rv7yrd/w...,deepwebteddy,Clippers5,[LAC] Michael Phelps,False,2022-01-03,2022-01-03 18:47:46.692418,-0.2960,0.136,0.864,0.000,0
4775,Yeah as long as he doesn’t hve a bad bout with...,14,https://www.reddit.com/r/nba/comments/rv7yrd/w...,Bigbadbuck,Nets3,Nets,False,2022-01-03,2022-01-03 18:47:46.692418,0.7269,0.103,0.587,0.311,1
4776,"yes i know, but madison square garden isn't in...",1,https://www.reddit.com/r/nba/comments/rv7yrd/w...,kikikza,Knicks4,:nyk-4: Knicks,False,2022-01-03,2022-01-03 18:47:46.692418,-0.5713,0.172,0.758,0.070,0


In [19]:
url = 'https://www.reddit.com/r/nba/comments/rsgss7/woman_hit_in_the_head_at_hornets_game_by_errant/'

In [13]:
submission = reddit.submission(url = 'https://www.reddit.com/r/nba/comments/rvbfqn/wojnarowski_theres_optimism_golden_warriors_guard/')

In [10]:
for top_level_comment in submission.comments:
    print(top_level_comment.body) # num comments may not match up with whats extracted bc overall count includes deleted, removed, or spam comments

She lives in North Carolina and this dude is offering her tickets to a Rockets game
She is trying to sue lol
MY NECK. MY BACK. MY NECK AND MY BACK.
Why does this always happen in charlotte
Shaq used to dive into the stands all the time. You know there is a  chance you might get hurt that's why they print it in back of your ticket. The team is not responsible.
She wants that check lmfao. Idk why she's acting like he did it on purpose.
There should be protective nets surrounding the court ! /s
It was an accident, dude offered you free tickets and would most likely say sorry next time he saw you. Woman seems to be milking this.
I do not blame her for being upset and looking for someone to blame. But it’s not on him, he’s just trying to ball. Social media response aside
I doubt anyone realized the severity of the incident when it happened. These things happen at sports games, usually without too much damage done. I can see why she is upset though, can't feel good to get hurt that badly and

AttributeError: 'MoreComments' object has no attribute 'body'

In [17]:
import pprint
pprint.pprint(vars(submission))

{'_comments': <praw.models.comment_forest.CommentForest object at 0x7fa7e80fb1c0>,
 '_comments_by_id': {'t1_hr4bvex': Comment(id='hr4bvex'),
                     't1_hr4bx5m': Comment(id='hr4bx5m'),
                     't1_hr4bxvr': Comment(id='hr4bxvr'),
                     't1_hr4bxwd': Comment(id='hr4bxwd'),
                     't1_hr4bxxv': Comment(id='hr4bxxv'),
                     't1_hr4byo0': Comment(id='hr4byo0'),
                     't1_hr4byvl': Comment(id='hr4byvl'),
                     't1_hr4bzt6': Comment(id='hr4bzt6'),
                     't1_hr4c11s': Comment(id='hr4c11s'),
                     't1_hr4c2vt': Comment(id='hr4c2vt'),
                     't1_hr4c38r': Comment(id='hr4c38r'),
                     't1_hr4c3pi': Comment(id='hr4c3pi'),
                     't1_hr4c4gt': Comment(id='hr4c4gt'),
                     't1_hr4c4lk': Comment(id='hr4c4lk'),
                     't1_hr4c5l5': Comment(id='hr4c5l5'),
                     't1_hr4c77g': Comment(id='

In [15]:
# we can only grab  the top 50 comments here
comment_list = []
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    print(vars(top_level_comment))

{'_replies': <praw.models.comment_forest.CommentForest object at 0x7fa77a027d90>, '_submission': Submission(id='rvbfqn'), '_reddit': <praw.reddit.Reddit object at 0x7fa77b9e9040>, 'subreddit_id': 't5_2qo4s', 'approved_at_utc': None, 'author_is_blocked': False, 'comment_type': None, 'awarders': [], 'mod_reason_by': None, 'banned_by': None, 'author_flair_type': 'richtext', 'total_awards_received': 0, 'subreddit': Subreddit(display_name='nba'), 'author_flair_template_id': 'ce02da8a-3feb-11e8-99d0-0e7ec6cafa10', 'likes': None, 'user_reports': [], 'saved': False, 'id': 'hr4c5l5', 'banned_at_utc': None, 'mod_reason_title': None, 'gilded': 0, 'archived': False, 'collapsed_reason_code': None, 'no_follow': False, 'author': Redditor(name='mnbnd'), 'can_mod_post': False, 'created_utc': 1641242542.0, 'send_replies': True, 'parent_id': 't3_rvbfqn', 'score': 261, 'author_fullname': 't2_18biczk5', 'approved_by': None, 'mod_note': None, 'all_awardings': [], 'collapsed': False, 'body': '"Golden Warrior

In [29]:
# we can only grab  the top 50 comments here
comment_list = []
score_list = []
url_list = []
submission.comments.replace_more(limit=0)
for comment in submission.comments.list():
    comment_list.append(comment.body)
    score_list.append(comment.score)
    url_list.append(url)

In [26]:
df2 = pd.DataFrame({'comment':comment_list, 'score':score_list, 'url':url,
                    'scrape_date': datetime.now().date(), 'scrape_ts': datetime.now()})

In [1]:
from utils import get_owner, get_ssm_parameter, aws_connection

df = get_owner('jacob')

df2 = get_ssm_parameter('jacobs_ssm_test')

df3 = get_ssm_parameter(5)

conn = aws_connection('nba_source')

The owner is jacob!
SSM Failed, Parameter validation failed:
Invalid type for parameter Name, value: 5, type: <class 'int'>, valid types: <class 'str'>
SQL Connection to nba_source Successful


In [16]:
def get_ssm_parameter(parameter_name: str) -> str:
    """
    Function to grab parameters from SSM
    
    note: withdecryption = false will make pg user not work bc its a securestring.
        ignored for String and StringList parameter types

    Args:
        parameter_name (string) - name of the parameter you want

    Returns:
        parameter_value (string)
    """
    try:
        ssm = boto3.client('ssm')
        resp = ssm.get_parameter(
            Name = parameter_name,
            WithDecryption=True 
        )
        return resp["Parameter"]["Value"]
    except BaseException as error:
        print(f'SSM Failed, {error}')
        df = []
        return df

In [18]:
df = get_ssm_parameter(5)

SSM Failed, Parameter validation failed:
Invalid type for parameter Name, value: 5, type: <class 'int'>, valid types: <class 'str'>


In [13]:
df = get_ssm_parameter('jacobs_ssm_test')
df2 = get_ssm_parameter('jacobs_ssm_pg_user')

In [5]:
def sql_connection():
    try:
        connection = create_engine('postgresql+psycopg2://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + '5432' + '/' + os.environ.get('RDS_DB'),
                                    connect_args = {'options': '-csearch_path=nba_source'}, # defining schema to connect to
                     echo = False)
        logging.info('SQL Connection Successful')
        print('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        logging.info('SQL Connection Failed, Error:', e)
        print('SQL Connection Failed, Error:', e)
        return(e)
conn = sql_connection()

SQL Connection Successful


In [6]:
schedule_df.to_sql(con = conn, name = "aws_schedule_source", if_exists = 'replace', index = False)

In [8]:
df = pd.read_sql('SELECT * FROM aws_reddit_comment_data_source;', conn)

In [9]:
df2 = df[['comment']]


In [10]:
df2.to_csv('reddit_comments.csv', index = False)

In [2]:
def schedule_scraper(month):
    try:
        global schedule_df
        url = "https://www.basketball-reference.com/leagues/NBA_2022_games-{}.html".format(month)
        html = urlopen(url)
        soup = BeautifulSoup(html)

        headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
        headers[6] = 'boxScoreLink'
        headers[7] = 'isOT'
        headers = headers[1:]

        rows = soup.findAll('tr')[1:]
        date_info = [[th.getText() for th in rows[i].findAll('th')]
                for i in range(len(rows))]

        game_info = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
        date_info = [i[0] for i in date_info]

        schedule = pd.DataFrame(game_info, columns = headers)
        schedule['Date'] = date_info

        schedule_df = schedule_df.append(schedule)
        logging.info(f'Schedule Function Completed for {month}, retrieving {len(schedule_df)} rows')
        print(f'Schedule Function Completed for {month}, retrieving {len(schedule_df)} rows')
    except ValueError:
        logging.info("Schedule Scraper Function Failed")
        print("Schedule Scraper Function Failed")
        df = []
        return(df)

month_list = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
schedule_df = pd.DataFrame()
for month in month_list:
    schedule_scraper(month)

Schedule Function Completed for october, retrieving 93 rows
Schedule Function Completed for november, retrieving 318 rows
Schedule Function Completed for december, retrieving 527 rows
Schedule Function Completed for january, retrieving 758 rows
Schedule Function Completed for february, retrieving 921 rows
Schedule Function Completed for march, retrieving 1150 rows
Schedule Function Completed for april, retrieving 1230 rows


In [3]:
schedule_df = schedule_df[['Start (ET)', 'Visitor/Neutral', 'Home/Neutral', 'Date']]
schedule_df['proper_date'] = pd.to_datetime(schedule_df['Date']).dt.date
schedule_df.columns = schedule_df.columns.str.lower()
schedule_df = schedule_df.rename(columns = {"start (et)": "start_time", "visitor/neutral": "away_team", "home/neutral": "home_team"})

In [4]:
schedule_df.to_sql(con = conn, name = "aws_schedule_source", if_exists = 'replace', index = False)

NameError: name 'conn' is not defined

In [3]:
def get_contracts(run_type: str):
    if run_type == 'dev':
        df = pd.read_html('https://www.basketball-reference.com/contracts/players.html', header = 1)[0]
        df = df.rename(columns={df.columns[2]: 'team', df.columns[3]: 'season_salary'})
        df = df[['Player', 'team', 'season_salary']]
        df.columns = df.columns.str.lower()
        df = df.drop_duplicates()
        df = df.query('season_salary != "Salary" & season_salary != "2021-22"').reset_index()
        df['season_salary'] = df['season_salary'].str.replace(',', "", regex = True)
        df['season_salary'] = df['season_salary'].str.replace('$', "", regex = True)
        df['team'] = df['team'].str.replace("PHO", "PHX")
        df['team'] = df['team'].str.replace("CHO", "CHA")
        df['team'] = df['team'].str.replace("BRK", "BKN")
        df['season_salary'] = pd.to_numeric(df['season_salary'])
        df['player'] = df['player'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')
        df['player'] = df['player'].str.replace(" Jr.", "", regex = True)
        df['player'] = df['player'].str.replace(" Sr.", "", regex = True)
        df['player'] = df['player'].str.replace(" II", "", regex = True)
        df['player'] = df['player'].str.replace(" III", "", regex = True)
        df['player'] = df['player'].str.replace(" IV", "", regex = True)
        df = df.reset_index(drop = True)
        df = df.sample(n = 5)
        return(df)
    else:
        df = pd.read_html('https://www.basketball-reference.com/contracts/players.html', header = 1)[0]
        df = df.rename(columns={df.columns[2]: 'team', df.columns[3]: 'season_salary'})
        df = df[['Player', 'team', 'season_salary']]
        df.columns = df.columns.str.lower()
        df = df.drop_duplicates()
        df = df.query('season_salary != "Salary" & season_salary != "2021-22"').reset_index()
        df['season_salary'] = df['season_salary'].str.replace(',', "", regex = True)
        df['season_salary'] = df['season_salary'].str.replace('$', "", regex = True)
        df['team'] = df['team'].str.replace("PHO", "PHX")
        df['team'] = df['team'].str.replace("CHO", "CHA")
        df['team'] = df['team'].str.replace("BRK", "BKN")
        df['season_salary'] = pd.to_numeric(df['season_salary'])
        df['player'] = df['player'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')
        df['player'] = df['player'].str.replace(" Jr.", "", regex = True)
        df['player'] = df['player'].str.replace(" Sr.", "", regex = True)
        df['player'] = df['player'].str.replace(" II", "", regex = True)
        df['player'] = df['player'].str.replace(" III", "", regex = True)
        df['player'] = df['player'].str.replace(" IV", "", regex = True)
        df = df.reset_index(drop = True)
        return(df)

In [6]:
contracts = get_contracts('dev')

In [16]:
class RunTypeError(BaseException):
    pass

In [17]:
def get_contracts(run_type: str):
    if run_type == 'dev':
        records_pct = .05
    elif run_type == 'prod':
        records_pct = 1
    else:
        raise RunTypeError(f"Please Select either dev or prod for run_type; you chose {run_type}")
    df = pd.read_html('https://www.basketball-reference.com/contracts/players.html', header = 1)[0]
    df = df.rename(columns={df.columns[2]: 'team', df.columns[3]: 'season_salary'})
    df = df[['Player', 'team', 'season_salary']]
    df.columns = df.columns.str.lower()
    df = df.drop_duplicates()
    df = df.query('season_salary != "Salary" & season_salary != "2021-22"').reset_index()
    df['season_salary'] = df['season_salary'].str.replace(',', "", regex = True)
    df['season_salary'] = df['season_salary'].str.replace('$', "", regex = True)
    df['team'] = df['team'].str.replace("PHO", "PHX")
    df['team'] = df['team'].str.replace("CHO", "CHA")
    df['team'] = df['team'].str.replace("BRK", "BKN")
    df['season_salary'] = pd.to_numeric(df['season_salary'])
    df['player'] = df['player'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')
    df['player'] = df['player'].str.replace(" Jr.", "", regex = True)
    df['player'] = df['player'].str.replace(" Sr.", "", regex = True)
    df['player'] = df['player'].str.replace(" II", "", regex = True)
    df['player'] = df['player'].str.replace(" III", "", regex = True)
    df['player'] = df['player'].str.replace(" IV", "", regex = True)
    df = df.reset_index(drop = True)
    df = df.sample(frac = records_pct)
    return(df)

In [19]:
contracts = get_contracts('prod')

In [10]:
contracts.to_sql(con = conn, name = "aws_contracts_source", if_exists = 'replace', index = False)

In [11]:
data.to_sql(con = conn, name = 'aws_odds_source', if_exists = 'append', index = False)

In [15]:
['index', 'player', 'team', 'season_salary'] == bby

True

In [9]:
contracts.to_sql(con = conn, name = "aws_contracts_source", if_exists = 'replace', index = False)

In [5]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022_preseason_odds.html'
df = pd.read_html(url)[0]
df

,Team,Odds,Unnamed: 2,W-L O/U,Result
0,Brooklyn Nets,230,NaN,56.5,0-1 (under)
1,Los Angeles Lakers,425,NaN,52.5,0-1 (under)
2,Milwaukee Bucks,850,NaN,54.5,1-0 (under)
3,Golden State Warriors,1100,NaN,48.5,1-0 (under)
4,Utah Jazz,1400,NaN,52.5,1-0 (under)
5,Phoenix Suns,1400,NaN,51.5,0-1 (under)
6,Philadelphia 76ers,1600,NaN,50.5,1-0 (under)
7,Los Angeles Clippers,1600,NaN,45.5,0-0 (under)
8,Denver Nuggets,2200,NaN,47.5,1-0 (under)
9,Dallas Mavericks,2800,NaN,48.5,0-0 (under)


In [8]:
df = df.rename(columns = {df.columns[3]: 'predicted'})
df.columns = df.columns.str.lower()
df = df[['team', 'odds', 'predicted']]

In [10]:
df.to_sql(con = conn, name = 'aws_preseason_odds_source', if_exists = 'replace', index = False)

In [5]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022.html'
df = pd.read_html(url)[5]

In [22]:
def get_opp_stats():
    try:
        url = 'https://www.basketball-reference.com/leagues/NBA_2022.html'
        df = pd.read_html(url)[5]
        df = df[['Team', 'FG%', '3P%', '3P', 'PTS']]
        df = df.rename(columns = {df.columns[0]: 'team', df.columns[1]: 'fg_percent_opp', df.columns[2]: 'threep_percent_opp',
                        df.columns[3]: 'threep_made_opp', df.columns[4]: 'ppg_opp'})
        df = df.query('team != "League Average"')
        df = df.reset_index(drop = True)
        return(df)
    except IndexError:
        logging.info("Opp Stats Function Failed for Today's Games")
        print("Opp Stats Function Failed for Today's Games")
        df = []
        return df


In [7]:
def get_odds():
    """
    Web Scrape function w/ pandas read_html that grabs current day's nba odds
    Args:
        None
    Returns:
        Pandas DataFrame of NBA moneyline + spread odds for upcoming games for that day
    """
    url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"

    try:
        df = pd.read_html(url)

        data1 = df[0].copy()
        data1.columns.values[0] = "Tomorrow"
        date_try = str(year) + " " + data1.columns[0]
        data1["date"] = np.where(
            date_try == "2021 Tomorrow",
            datetime.now().date(),  # if the above is true, then return this
            str(year) + " " + data1.columns[0],  # if false then return this
        )
        date_try = data1["date"].iloc[0]
        data1.reset_index(drop=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace(
            "LA Clippers", "LAC Clippers", regex=True
        )

        data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
        data1["Time"] = data1["Tomorrow"].str.split().str[0]
        data1["datetime1"] = (
            pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
            - timedelta(hours=6)
            + timedelta(days=1)
        )
        data = data1.reset_index(drop=True)
        data["SPREAD"] = data["SPREAD"].str[:-4]
        data["TOTAL"] = data["TOTAL"].str[:-4]
        data["TOTAL"] = data["TOTAL"].str[2:]
        data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
        data["Tomorrow"] = pd.DataFrame(
            [
                str(line).strip("[").strip("]").replace("'", "")
                for line in data["Tomorrow"]
            ]
        )
        data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
        data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
        data.columns = data.columns.str.lower()
        data = data[["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]]
        data = data.rename(columns={data.columns[0]: "team"})
        data = data.query("date == date.min()")  # only grab games from upcoming day
        print(f"Odds Function Successful, retrieving {len(data)} rows")
        logging.info(f"Odds Function Successful, retrieving {len(data)} rows")
        return data
    except BaseException as error:
        print(f"Odds Function Failed, {error}")
        logging.info(f"Odds Function Failed, {error}")
        data = []
        return data

In [15]:
def get_odds():
    """
    Web Scrape function w/ pandas read_html that grabs current day's nba odds
    Args:
        None
    Returns:
        Pandas DataFrame of NBA moneyline + spread odds for upcoming games for that day
    """
    try:
        url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"
        df = pd.read_html(url)

        data1 = df[0].copy()
        data1.columns.values[0] = "Tomorrow"
        date_try = str(year) + " " + data1.columns[0]
        data1["date"] = np.where(
            date_try == "2021 Tomorrow",
            datetime.now().date(),  # if the above is true, then return this
            str(year) + " " + data1.columns[0],  # if false then return this
        )
        # )
        date_try = data1["date"].iloc[0]
        data1.reset_index(drop=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace(
            "LA Clippers", "LAC Clippers", regex=True
        )

        data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
        data1["Time"] = data1["Tomorrow"].str.split().str[0]
        data1["datetime1"] = (
            pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
            - timedelta(hours=6)
            + timedelta(days=1)
        )
        if len(df) > 1:
            data2 = df[1].copy()
            data2.columns.values[0] = "Tomorrow"
            data2.reset_index(drop=True)
            data2["Tomorrow"] = data2["Tomorrow"].str.replace(
                "LA Clippers", "LAC Clippers", regex=True
            )
            data2["Tomorrow"] = data2["Tomorrow"].str.replace("AM", "AM ", regex=True)
            data2["Tomorrow"] = data2["Tomorrow"].str.replace("PM", "PM ", regex=True)
            data2["Time"] = data2["Tomorrow"].str.split().str[0]
            data2["datetime1"] = (
                pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data2["Time"])
                - timedelta(hours=6)
                + timedelta(days=1)
            )
            data2["date"] = data2["datetime1"].dt.date

            data = data1.append(data2).reset_index(drop=True)
            data["SPREAD"] = data["SPREAD"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[2:]
            data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
            data["Tomorrow"] = pd.DataFrame(
                [
                    str(line).strip("[").strip("]").replace("'", "")
                    for line in data["Tomorrow"]
                ]
            )
            data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
            data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
            data.columns = data.columns.str.lower()
            data = data[["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]]
            data = data.rename(columns={data.columns[0]: "team"})
            data = data.query("date == date.min()")  # only grab games from upcoming day
            print(f"Odds Function Successful, retrieving {len(data)} rows")
            logging.info(f"Odds Function Successful, retrieving {len(data)} rows")
            return data
        else:
            data = data1.reset_index(drop=True)
            data["SPREAD"] = data["SPREAD"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[2:]
            data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
            data["Tomorrow"] = pd.DataFrame(
                [
                    str(line).strip("[").strip("]").replace("'", "")
                    for line in data["Tomorrow"]
                ]
            )
            data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
            data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
            data.columns = data.columns.str.lower()
            data = data[["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]]
            data = data.rename(columns={data.columns[0]: "team"})
            data = data.query("date == date.min()")  # only grab games from upcoming day
            print(f"Odds Function Successful, retrieving {len(data)} rows")
            logging.info(f"Odds Function Successful, retrieving {len(data)} rows")
            return data
    except BaseException as error:
        print(f"Odds Function Failed, {error}")
        logging.info(f"Odds Function Failed, {error}")
        data = []
        return data


In [ ]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"

In [4]:
def get_odds():
    """
    Web Scrape function w/ pandas read_html that grabs current day's nba odds

    Args:
        None
    Returns:
        Pandas DataFrame of NBA moneyline + spread odds for upcoming games for that day
    """
    try:
        url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"
        df = pd.read_html(url)

        data1 = df[0].copy()
        data1.columns.values[0] = "Tomorrow"
        date_try = str(year) + " " + data1.columns[0]
        data1["date"] = np.where(
            date_try == "2021 Tomorrow",
            datetime.now().date(),  # if the above is true, then return this
            str(year) + " " + data1.columns[0],  # if false then return this
        )
        # )
        date_try = data1["date"].iloc[0]
        data1.reset_index(drop=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace(
            "LA Clippers", "LAC Clippers", regex=True
        )

        data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
        data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
        data1["Time"] = data1["Tomorrow"].str.split().str[0]
        data1["datetime1"] = (
            pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
            - timedelta(hours=6)
            + timedelta(days=1)
        )
        if len(df) > 1:  # if more than 1 day's data appears then do this
            data2 = df[1].copy()
            data2.columns.values[0] = "Tomorrow"
            data2.reset_index(drop=True)
            data2["Tomorrow"] = data2["Tomorrow"].str.replace(
                "LA Clippers", "LAC Clippers", regex=True
            )
            data2["Tomorrow"] = data2["Tomorrow"].str.replace("AM", "AM ", regex=True)
            data2["Tomorrow"] = data2["Tomorrow"].str.replace("PM", "PM ", regex=True)
            data2["Time"] = data2["Tomorrow"].str.split().str[0]
            data2["datetime1"] = (
                pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data2["Time"])
                - timedelta(hours=6)
                + timedelta(days=1)
            )
            data2["date"] = data2["datetime1"].dt.date

            data = data1.append(data2).reset_index(drop=True)
            data["SPREAD"] = data["SPREAD"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[2:]
            data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
            data["Tomorrow"] = pd.DataFrame(
                [
                    str(line).strip("[").strip("]").replace("'", "")
                    for line in data["Tomorrow"]
                ]
            )
            data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
            data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
            data.columns = data.columns.str.lower()
            data = data[
                ["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]
            ]
            data = data.rename(columns={data.columns[0]: "team"})
            data = data.query("date == date.min()")  # only grab games from upcoming day
            print(
                f"Odds Function Successful for {len(df)} day, retrieving {len(data)} rows"
            )
            logging.info(
                f"Odds Function Successful {len(df)} day, retrieving {len(data)} rows"
            )
            return data
        else:  # if there's only 1 day of data then just use that
            data = data1.reset_index(drop=True)
            data["SPREAD"] = data["SPREAD"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[:-4]
            data["TOTAL"] = data["TOTAL"].str[2:]
            data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
            data["Tomorrow"] = pd.DataFrame(
                [
                    str(line).strip("[").strip("]").replace("'", "")
                    for line in data["Tomorrow"]
                ]
            )
            data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
            data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
            data.columns = data.columns.str.lower()
            data = data[
                ["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]
            ]
            data = data.rename(columns={data.columns[0]: "team"})
            data = data.query("date == date.min()")  # only grab games from upcoming day
            print(
                f"Odds Function Successful for {len(df)} day, retrieving {len(data)} rows"
            )
            logging.info(
                f"Odds Function Successful {len(df)} day, retrieving {len(data)} rows"
            )
            return data
    except BaseException as error:
        print(f"Odds Function Failed for {len(df)} day, {error}")
        logging.info(f"Odds Function Failed {len(df)} day, {error}")
        data = []
        return data

In [7]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"
df = pd.read_html(url)

data1 = df[0].copy()
data1.columns.values[0] = "Tomorrow"
date_try = str(year) + " " + data1.columns[0]
data1["date"] = np.where(
    date_try == "2021 Tomorrow",
    datetime.now().date(),  # if the above is true, then return this
    str(year) + " " + data1.columns[0],  # if false then return this
)
# )
date_try = data1["date"].iloc[0]
data1.reset_index(drop=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace(
    "LA Clippers", "LAC Clippers", regex=True
)

data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
data1["Time"] = data1["Tomorrow"].str.split().str[0]
data1["datetime1"] = (
    pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
    - timedelta(hours=6)
    + timedelta(days=1)
)
if len(df) > 1:  # if more than 1 day's data appears then do this
    data2 = df[1].copy()
    data2.columns.values[0] = "Tomorrow"
    data2.reset_index(drop=True)
    data2["Tomorrow"] = data2["Tomorrow"].str.replace(
        "LA Clippers", "LAC Clippers", regex=True
    )
    data2["Tomorrow"] = data2["Tomorrow"].str.replace("AM", "AM ", regex=True)
    data2["Tomorrow"] = data2["Tomorrow"].str.replace("PM", "PM ", regex=True)
    data2["Time"] = data2["Tomorrow"].str.split().str[0]
    data2["datetime1"] = (
        pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data2["Time"])
        - timedelta(hours=6)
        + timedelta(days=1)
    )
    data2["date"] = data2["datetime1"].dt.date

    data = data1.append(data2).reset_index(drop=True)
    data["SPREAD"] = data["SPREAD"].str[:-4]
    data["TOTAL"] = data["TOTAL"].str[:-4]
    data["TOTAL"] = data["TOTAL"].str[2:]
    data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
    data["Tomorrow"] = pd.DataFrame(
        [
            str(line).strip("[").strip("]").replace("'", "")
            for line in data["Tomorrow"]
        ]
    )
    data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
    data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
    data.columns = data.columns.str.lower()
    data = data[
        ["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]
    ]
    data = data.rename(columns={data.columns[0]: "team"})
    data = data.query("date == date.min()")  # only grab games from upcoming day
else:  # if there's only 1 day of data then just use that
    data = data1.reset_index(drop=True)
    data["SPREAD"] = data["SPREAD"].str[:-4]
    data["TOTAL"] = data["TOTAL"].str[:-4]
    data["TOTAL"] = data["TOTAL"].str[2:]
    data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
    data["Tomorrow"] = pd.DataFrame(
        [
            str(line).strip("[").strip("]").replace("'", "")
            for line in data["Tomorrow"]
        ]
    )
    data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
    data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
    data.columns = data.columns.str.lower()
    data = data[
        ["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]
    ]
    data = data.rename(columns={data.columns[0]: "team"})
    data = data.query("date == date.min()")  # only grab games from upcoming day

In [5]:
df = get_odds()

Odds Function Successful for 2 day, retrieving 18 rows


In [8]:
data['date'] = data['date'] + timedelta(days = 1)

In [9]:
data['datetime1'] = data['datetime1'] + timedelta(days = 1)

In [14]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"
df = pd.read_html(url)
# df2 = df.copy()[1]

# for i in df:
#     df2 = df.copy()[i]

if len(df) > 1:
    print('hio')
else:
    print('fuq')

fuq


In [3]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846?category=game-lines&subcategory=game"
df = pd.read_html(url)

data1 = df[0].copy()
data1.columns.values[0] = "Tomorrow"
date_try = str(year) + " " + data1.columns[0]
data1["date"] = np.where(
    date_try == "2021 Tomorrow",
    datetime.now().date(),  # if the above is true, then return this
    str(year) + " " + data1.columns[0],  # if false then return this
)
date_try = data1["date"].iloc[0]
data1.reset_index(drop=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace(
    "LA Clippers", "LAC Clippers", regex=True
)

data1["Tomorrow"] = data1["Tomorrow"].str.replace("AM", "AM ", regex=True)
data1["Tomorrow"] = data1["Tomorrow"].str.replace("PM", "PM ", regex=True)
data1["Time"] = data1["Tomorrow"].str.split().str[0]
data1["datetime1"] = (
    pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data1["Time"])
    - timedelta(hours=6)
    + timedelta(days=1)
)
data = data1.reset_index(drop=True)
data["SPREAD"] = data["SPREAD"].str[:-4]
data["TOTAL"] = data["TOTAL"].str[:-4]
data["TOTAL"] = data["TOTAL"].str[2:]
data["Tomorrow"] = data["Tomorrow"].str.split().str[1:2]
data["Tomorrow"] = pd.DataFrame(
    [
        str(line).strip("[").strip("]").replace("'", "")
        for line in data["Tomorrow"]
    ]
)
data["SPREAD"] = data["SPREAD"].str.replace("pk", "-1", regex=True)
data["SPREAD"] = data["SPREAD"].str.replace("+", "", regex=True)
data.columns = data.columns.str.lower()
data = data[["tomorrow", "spread", "total", "moneyline", "date", "datetime1"]]
data = data.rename(columns={data.columns[0]: "team"})
data = data.query("date == date.min()")
# data2 = df[1].copy()
# data2.columns.values[0] = "Tomorrow"
# data2.reset_index(drop=True)
# data2["Tomorrow"] = data2["Tomorrow"].str.replace(
#     "LA Clippers", "LAC Clippers", regex=True
# )
# data2["Tomorrow"] = data2["Tomorrow"].str.replace("AM", "AM ", regex=True)
# data2["Tomorrow"] = data2["Tomorrow"].str.replace("PM", "PM ", regex=True)
# data2["Time"] = data2["Tomorrow"].str.split().str[0]
# data2["datetime1"] = (
#     pd.to_datetime(date_try.strftime("%Y-%m-%d") + " " + data2["Time"])
#     - timedelta(hours=6)
#     + timedelta(days=1)
# )
# data2["date"] = data2["datetime1"].dt.date

In [7]:
df

[                  Tomorrow     SPREAD        TOTAL  MONEYLINE
 0          12:10AMBKN Nets     -6-105  O 208.5-105     -240.0
 1     12:10AMCLE Cavaliers     +6-115  U 208.5-115      195.0
 2       12:10AMCHA Hornets   +4.5-110  O 216.5-110      160.0
 3       12:10AMWAS Wizards   -4.5-110  U 216.5-110     -190.0
 4       12:40AMOKC Thunder  +11.5-105    O 213-110      475.0
 5         12:40AMATL Hawks  -11.5-115    U 213-110     -675.0
 6       12:40AMHOU Rockets    +12-115  O 213.5-110      500.0
 7       12:40AMBOS Celtics    -12-105  U 213.5-110     -720.0
 8         1:10AMIND Pacers     +2-115  O 212.5-115      105.0
 9          1:10AMCHI Bulls     -2-105  U 212.5-105     -125.0
 10         1:10AMORL Magic    +13-105  O 218.5-110      650.0
 11         1:10AMMIL Bucks    -13-115  U 218.5-110    -1000.0
 12  1:10AMMIN Timberwolves   -3.5-110    O 214-110        NaN
 13       1:10AMNO Pelicans   +3.5-110    U 214-110        NaN
 14          1:40AMPHO Suns   -5.5-110    O 221-115    

In [3]:
df = get_odds()

Odds Function Failed, list index out of range


In [3]:
def get_boxscores(month=month, day=day, year=year):
    """
    Function that grabs box scores from a given date in mmddyyyy format - defaults to yesterday.  values can be ex. 1 or 01
    Args:
        month (string) - month value of the game played (0 - 12)
        day (string) - day value of the game played (1 - 31)
        year (string) - year value of the game played (2021)
    Returns:
        Pandas DataFrame of Player Aggregate Season stats
    """
    url = "https://www.basketball-reference.com/friv/dailyleaders.fcgi?month={}&day={}&year={}&type=all".format(
        month, day, year
    )

    try:
        html = urlopen(url)
        soup = BeautifulSoup(html, "html.parser")
        headers = [th.getText() for th in soup.findAll("tr", limit=2)[0].findAll("th")]
        headers = headers[1:]
        headers[1] = "Team"
        headers[2] = "Location"
        headers[3] = "Opponent"
        headers[4] = "Outcome"
        headers[6] = "FGM"
        headers[8] = "FGPercent"
        headers[9] = "threePFGMade"
        headers[10] = "threePAttempted"
        headers[11] = "threePointPercent"
        headers[14] = "FTPercent"
        headers[15] = "OREB"
        headers[16] = "DREB"
        headers[24] = "PlusMinus"

        rows = soup.findAll("tr")[1:]
        player_stats = [
            [td.getText() for td in rows[i].findAll("td")] for i in range(len(rows))
        ]

        df = pd.DataFrame(player_stats, columns=headers)
        df[
            [
                "FGM",
                "FGA",
                "FGPercent",
                "threePFGMade",
                "threePAttempted",
                "threePointPercent",
                "OREB",
                "DREB",
                "TRB",
                "AST",
                "STL",
                "BLK",
                "TOV",
                "PF",
                "PTS",
                "PlusMinus",
                "GmSc",
            ]
        ] = df[
            [
                "FGM",
                "FGA",
                "FGPercent",
                "threePFGMade",
                "threePAttempted",
                "threePointPercent",
                "OREB",
                "DREB",
                "TRB",
                "AST",
                "STL",
                "BLK",
                "TOV",
                "PF",
                "PTS",
                "PlusMinus",
                "GmSc",
            ]
        ].apply(
            pd.to_numeric
        )
        df["date"] = str(year) + "-" + str(month) + "-" + str(day)
        df["date"] = pd.to_datetime(df["date"])
        df["Type"] = season_type
        df["Season"] = 2022
        df["Location"] = df["Location"].apply(lambda x: "A" if x == "@" else "H")
        df["Team"] = df["Team"].str.replace("PHO", "PHX")
        df["Team"] = df["Team"].str.replace("CHO", "CHA")
        df["Team"] = df["Team"].str.replace("BRK", "BKN")
        df["Opponent"] = df["Opponent"].str.replace("PHO", "PHX")
        df["Opponent"] = df["Opponent"].str.replace("CHO", "CHA")
        df["Opponent"] = df["Opponent"].str.replace("BRK", "BKN")
        df = df.query("Player == Player").reset_index(drop=True)
        df["Player"] = (
            df["Player"]
            .str.normalize("NFKD")
            .str.encode("ascii", errors="ignore")
            .str.decode("utf-8")
        )
        df.columns = df.columns.str.lower()
        logging.info(
            f"Box Score Function Successful, retrieving {len(df)} rows for {year}-{month}-{day}"
        )
        print(
            f"Box Score Function Successful, retrieving {len(df)} rows for {year}-{month}-{day}"
        )
        return df
    except BaseException as error:
        logging.info(
            f"Box Score Function Failed, {error}, no data available for {year}-{month}-{day}"
        )
        print(
            f"Box Score Function Failed, {error}, no data available for {year}-{month}-{day}"
        )
        df = []
        return df


In [28]:
boxscores = get_boxscores()

Box Score Function Successful, retrieving 237 rows for 2021-11-12


In [62]:
def get_pbp_data(df):
    """
    Web Scrape function w/ pandas read_html that uses aliases via boxscores function
    to scrape the pbp data interactively for each game played the previous day
    Args:
        df (Pandas DataFrame) - the DataFrame result from running the boxscores function.
    Returns:
        All PBP Data for the games in the input df
    """
    try:
        if len(df) > 0:
            yesterday_hometeams = (
                df.query('location == "H"')[["team"]].drop_duplicates().dropna()
            )
            yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
                "PHX", "PHO"
            )
            yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
                "CHA", "CHO"
            )
            yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
                "BKN", "BRK"
            )

            away_teams = (
                df.query('location == "A"')[["team", "opponent"]]
                .drop_duplicates()
                .dropna()
            )
            away_teams = away_teams.rename(
                columns={
                    away_teams.columns[0]: "AwayTeam",
                    away_teams.columns[1]: "HomeTeam",
                }
            )
        else:
            yesterday_hometeams = []

        if len(yesterday_hometeams) > 0:
            try:
                newdate = str(
                    df["date"].drop_duplicates()[0].date()
                )  # this assumes all games in the boxscores df are 1 date
                newdate = pd.to_datetime(newdate).strftime(
                    "%Y%m%d"
                )  # formatting into url format.
                pbp_list = pd.DataFrame()
                for i in yesterday_hometeams["team"]:
                    url = "https://www.basketball-reference.com/boxscores/pbp/{}0{}.html".format(
                        newdate, i
                    )
                    df = pd.read_html(url)[0]
                    df.columns = df.columns.map("".join)
                    df = df.rename(
                        columns={
                            df.columns[0]: "Time",
                            df.columns[1]: "descriptionPlayVisitor",
                            df.columns[2]: "AwayScore",
                            df.columns[3]: "Score",
                            df.columns[4]: "HomeScore",
                            df.columns[5]: "descriptionPlayHome",
                        }
                    )
                    conditions = [
                        (
                            df["HomeScore"].str.contains("Jump ball:", na=False)
                            & df["Time"].str.contains("12:00.0")
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 2nd quarter", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 3rd quarter", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 4th quarter", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 1st overtime", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 2nd overtime", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 3rd overtime", na=False
                            )
                        ),
                        (
                            df["HomeScore"].str.contains(
                                "Start of 4th overtime", na=False
                            )
                        ),  # if more than 4 ots then rip
                    ]
                    values = [
                        "1st Quarter",
                        "2nd Quarter",
                        "3rd Quarter",
                        "4th Quarter",
                        "1st OT",
                        "2nd OT",
                        "3rd OT",
                        "4th OT",
                    ]
                    df["Quarter"] = np.select(conditions, values, default=None)
                    df["Quarter"] = df["Quarter"].fillna(method="ffill")
                    df = df.query(
                        'Time != "Time" & Time != "2nd Q" & Time != "3rd Q" & Time != "4th Q" & Time != "1st OT" & Time != "2nd OT" & Time != "3rd OT" & Time != "4th OT"'
                    ).copy()  # use COPY to get rid of the fucking goddamn warning bc we filtered stuf out
                    # anytime you filter out values w/o copying and run code like the lines below it'll throw a warning.
                    df["HomeTeam"] = i
                    df["HomeTeam"] = df["HomeTeam"].str.replace("PHO", "PHX")
                    df["HomeTeam"] = df["HomeTeam"].str.replace("CHO", "CHA")
                    df["HomeTeam"] = df["HomeTeam"].str.replace("BRK", "BKN")
                    df = df.merge(away_teams)
                    df[["scoreAway", "scoreHome"]] = df["Score"].str.split(
                        "-", expand=True, n = 1
                    )
                    df["scoreAway"] = pd.to_numeric(df["scoreAway"], errors="coerce")
                    df["scoreAway"] = df["scoreAway"].fillna(method="ffill")
                    df["scoreAway"] = df["scoreAway"].fillna(0)
                    df["scoreHome"] = pd.to_numeric(df["scoreHome"], errors="coerce")
                    df["scoreHome"] = df["scoreHome"].fillna(method="ffill")
                    df["scoreHome"] = df["scoreHome"].fillna(0)
                    df["marginScore"] = df["scoreHome"] - df["scoreAway"]
                    df["Date"] = yesterday
                    df = df.rename(
                        columns={
                            df.columns[0]: "timeQuarter",
                            df.columns[6]: "numberPeriod",
                        }
                    )
                    pbp_list = pbp_list.append(df)
                    df = pd.DataFrame()
                pbp_list.columns = pbp_list.columns.str.lower()
                pbp_list = pbp_list.query(
                    "(awayscore.notnull()) | (homescore.notnull())", engine="python"
                )
                logging.info(
                    f"PBP Function Successful, retrieving {len(pbp_list)} rows for {year}-{month}-{day}"
                )
                print(
                    f"Box Score Function Successful, retrieving {len(pbp_list)} rows for {year}-{month}-{day}"
                )
                # filtering only scoring plays here, keep other all other rows in future for lineups stuff etc.
                return pbp_list
            except BaseException as error:
                logging.info(f"PBP Transformation Logic Failed for {i}, {error}")
                print(f"PBP Transformation Logic Failed for {i}, {error}")
                df = []
                return df
        else:
            df = []
            logging.info("PBP Function No Data Yesterday")
            print("PBP Function No Data Yesterday")
            return df
    except BaseException as error:
        logging.info(f"PBP Function Failed, {error}")
        print(f"PBP Function Failed, {error}")
        data = []
        return data

In [63]:
pbp = get_pbp_data(boxscores)

Box Score Function Successful, retrieving 1270 rows for 2021-11-12


In [8]:
yesterday_hometeams = (
    df.query('location == "H"')[["team"]].drop_duplicates().dropna()
)
yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
    "PHX", "PHO"
)
yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
    "CHA", "CHO"
)
yesterday_hometeams["team"] = yesterday_hometeams["team"].str.replace(
    "BKN", "BRK"
)

away_teams = (
    df.query('location == "A"')[["team", "opponent"]]
    .drop_duplicates()
    .dropna()
)
away_teams = away_teams.rename(
    columns={
        away_teams.columns[0]: "AwayTeam",
        away_teams.columns[1]: "HomeTeam",
    }
)

newdate = str(
    df["date"].drop_duplicates()[0].date()
)  # this assumes all games in the boxscores df are 1 date
newdate = pd.to_datetime(newdate).strftime(
    "%Y%m%d"
)  # formatting into url format.
pbp_list = pd.DataFrame()

In [35]:
url = 'https://www.basketball-reference.com/boxscores/pbp/202111120OKC.html'
df2 = pd.read_html(url)[0]

In [52]:
df = df2.copy()
df.columns = df.columns.map("".join)
df = df.rename(
    columns={
        df.columns[0]: "Time",
        df.columns[1]: "descriptionPlayVisitor",
        df.columns[2]: "AwayScore",
        df.columns[3]: "Score",
        df.columns[4]: "HomeScore",
        df.columns[5]: "descriptionPlayHome",
    }
)
conditions = [
    (
        df["HomeScore"].str.contains("Jump ball:", na=False)
        & df["Time"].str.contains("12:00.0")
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 2nd quarter", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 3rd quarter", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 4th quarter", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 1st overtime", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 2nd overtime", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 3rd overtime", na=False
        )
    ),
    (
        df["HomeScore"].str.contains(
            "Start of 4th overtime", na=False
        )
    ),  # if more than 4 ots then rip
]
values = [
    "1st Quarter",
    "2nd Quarter",
    "3rd Quarter",
    "4th Quarter",
    "1st OT",
    "2nd OT",
    "3rd OT",
    "4th OT",
]
df["Quarter"] = np.select(conditions, values, default=None)
df["Quarter"] = df["Quarter"].fillna(method="ffill")
df = df.query(
    'Time != "Time" & Time != "2nd Q" & Time != "3rd Q" & Time != "4th Q" & Time != "1st OT" & Time != "2nd OT" & Time != "3rd OT" & Time != "4th OT"'
).copy()  # use COPY to get rid of the fucking goddamn warning bc we filtered stuf out
# # anytime you filter out values w/o copying and run code like the lines below it'll throw a warning.
df["HomeTeam"] = 'OKC'
df["HomeTeam"] = df["HomeTeam"].str.replace("PHO", "PHX")
df["HomeTeam"] = df["HomeTeam"].str.replace("CHO", "CHA")
df["HomeTeam"] = df["HomeTeam"].str.replace("BRK", "BKN")
df = df.merge(away_teams)
df["Score"] = np.where(
    df["Score"] == str.contains("Jump ball:"),
    df["Score"].str.replace("-", " "),  # if the above is true, then return this
    df["Score"],  # if false then return this
)
df[["scoreAway", "scoreHome"]] = df["Score"].str.split(
    "-", expand=True, n = 1
)
# df["scoreAway"] = pd.to_numeric(df["scoreAway"], errors="coerce")
# df["scoreAway"] = df["scoreAway"].fillna(method="ffill")
# df["scoreAway"] = df["scoreAway"].fillna(0)
# df["scoreHome"] = pd.to_numeric(df["scoreHome"], errors="coerce")
# df["scoreHome"] = df["scoreHome"].fillna(method="ffill")
# df["scoreHome"] = df["scoreHome"].fillna(0)
# df["marginScore"] = df["scoreHome"] - df["scoreAway"]
# df["Date"] = yesterday
# df = df.rename(
#     columns={
#         df.columns[0]: "timeQuarter",
#         df.columns[6]: "numberPeriod",
#     }
# )

# # pbp_list2 = pbp_list2.append(df)
# # df = pd.DataFrame()
# pbp_list2.columns = pbp_list2.columns.str.lower()
# pbp_list2 = pbp_list2.query(
# "(awayscore.notnull()) | (homescore.notnull())", engine="python"
# )

AttributeError: type object 'str' has no attribute 'contains'

In [61]:
df = pd.DataFrame({"score": ["Jump ball: Shai-Gilgeous Alexander vs Jeremiah Robinson-Earl", "0-0"]})

# this doesnt work anymore because of a player with a hyphen pop up,
# which makes this return more than the 2 columns i want

df[["scoreAway", "scoreHome"]] = df["score"].str.split(
    "-", expand=True, n =1
)

In [23]:
pbp_list2 = df.copy()

In [6]:
pbp = get_pbp_data(df)

PBP Transformation Logic Failed, Columns must be same length as key


In [3]:
bucket='jacobsbucket97'
file_name='prac_boxscores'
date=datetime.now().date()

In [16]:
wr.s3.to_parquet(
    df = df,
    path = f"s3://{bucket}/boxscores/{file_name}-{date}.parquet",
    index = False
)
# make the initial bucket like nba_elt_project

{'paths': ['s3://jacobsbucket97/boxscores/prac_boxscores-2021-10-27.parquet'],
 'partitions_values': {}}

In [19]:
def write_to_s3(file_type, df, bucket = os.environ.get('s3_bucket')):
    date = datetime.now().date()
    # the date of the data, not the current date
    try:
        wr.s3.to_parquet(
            df = df,
            path = f"s3://{bucket}/{file_type}/{file_type}-{date}.parquet",
            index = False
        )
        print(f"Storing", len(df), file_type, "rows to S3 (s3://" + bucket + "/" + file_type +
                    "/" + file_type + "-" + str(date) + ")")
        logging.info(f"Storing", len(df), file_type, "rows to S3 (s3://" + bucket + "/" + file_type +
                    "/" + file_type + "-" + str(date) + ")")
        pass
    except BaseException as error:
        logging.info(f"S3 Storage Function Failed {file_type}, {error}")
        print(f"S3 Storage Function Failed for {file_type}, {error}")
        pass


In [20]:
write_to_s3(file_type = 'boxscores',
            df = df)

Storing 123 boxscores rows to S3 (s3://jacobsbucket97/boxscores/boxscores-2021-11-06)


In [17]:
new_df = wr.s3.read_parquet('s3://jacobsbucket97/boxscores/prac_boxscores-2021-10-27.parquet')